In [ ]:
import numpy as np
from view_sampler import *
from algs import *

import loss_funcs
from evaluate import eval_funcs

from evaluate.eval_log import EvalLog
from evaluate.evaluator import Evaluator
from evaluate.dataset import Dataset
from utils.visualize import *

import mealpy

plotter = SearchPlotter(update_freq=100, history=None, alpha=0.5)

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.3), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/hammer/world.xml", cam_config, simulation_time=0)
sim_viewer = ViewSampler("data/hammer/world_sim.xml", cam_config)

loss_func = loss_funcs.IOU()

algorithms = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.OriginalPSO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.MSA.OriginalMSA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.SCSO.OriginalSCSO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.OriginalSA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EVO.OriginalEVO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EFO.DevEFO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EO.ModifiedEO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.ICA.OriginalICA(revolution_prob=0.4, empire_count=7)),
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.FBIO.DevFBIO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.SARO.OriginalSARO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.GA.BaseGA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.CRO.OCRO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.DE.OriginalDE()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.PSS.OriginalPSS()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.SCA.DevSCA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.HC.OriginalHC()),
]

from mealpy.tuner import Tuner

eval_func = eval_funcs.XorDiff(0.1)
evaluator = Evaluator(world_viewer, sim_viewer, eval_func=eval_func)

dataset = Dataset.create_random(location=(0, 1.3, 0.3), num_samples=100, seed=1)
# dataset.visualize(sim_viewer)
run_config = MealRunConfig(time_limit=15, silent=True)

for alg in algorithms:
    log = EvalLog(alg)
    eval_losses = evaluator.evaluate(alg, run_config, dataset, log=None, plotter=None)
    log.save("logs/")
    print(f"{alg.get_name()} {alg.loss_func.get_name()}: {eval_losses}")

In [ ]:
world_viewer.close()
sim_viewer.close()
plotter.close()